#Imports

In [1]:
#@title
import sys
import pandas as pd
pd.options.mode.chained_assignment = None
pd.set_option('display.max_columns', None)
import numpy as np
!git clone https://github.com/bosemessi/StatsbombOpenData --q
from tqdm import tqdm
import requests 
import warnings
warnings.filterwarnings('ignore')

#Mount Drive

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#List the data files

In [3]:
ls /content/drive/MyDrive/Norwich/*.parquet

/content/drive/MyDrive/Norwich/defensivemetrics.parquet
/content/drive/MyDrive/Norwich/passcarry.parquet
/content/drive/MyDrive/Norwich/PassingAngle.parquet
/content/drive/MyDrive/Norwich/PassingAnglePressured.parquet
/content/drive/MyDrive/Norwich/time.parquet
/content/drive/MyDrive/Norwich/VAEP.parquet
/content/drive/MyDrive/Norwich/xGBC.parquet
/content/drive/MyDrive/Norwich/xP.parquet
/content/drive/MyDrive/Norwich/xT.parquet


#Read Expected Pass Completion Data File

In [6]:
xpdf = pd.read_parquet(prefix+'xP.parquet')
xpdf = xpdf[['name','Open Play Passes','Pass Completed Above Expected']]
xpdf.head()

,name,Open Play Passes,Pass Completed Above Expected
0,Aaron Mooy,184,0.962773
1,Abdalla Mahmoud El Said Bekhit,138,0.962087
2,Abdiel Arroyo Molinar,5,1.289162
3,Abdullah Ibrahim Al Maiouf,29,0.798129
4,Abdullah Ibrahim Otayf,252,1.014388


#Extract the defenders

In [5]:
#@title

prefix = '/content/drive/MyDrive/Norwich/'
df = pd.read_parquet('/content/StatsbombOpenData/WC2018.parquet').reset_index(drop=True)
defenders = df[df.position_name.isin(['Left Center Back','Right Center Back',
                                      'Center Back'])].player_name.unique()
flagnames = ['Francisco Javier Calvo Quesada','Joshua Kimmich',
             'Luis Carlos Tejada Hansell',
             'Michael Lang','Nicolás Alejandro Tagliafico',
             'Gabriel Iván Mercado','Hörður Björgvin Magnússon','Birkir Már Sævarsson',
             'Fedor Kudryashov','Éver Maximiliano David Banega','Edson Omar Álvarez Velázquez',
             'Marcus Rashford', 'İlkay Gündoğan', 'Dylan Bronn']
defenders = list(set(defenders) - set(flagnames))

#Aggregrate the defensive stats

In [13]:
#@title

dmdf = pd.read_parquet(prefix+'defensivemetrics.parquet')
dmdf.head()
for cols in ['Tackles','True Interceptions','Pressures','Successful Pressures', 'Defensive Acts']:
    dmdf['Padj_'+cols] = 2.0*dmdf[cols]/(1.0 + np.exp(-0.1*(dmdf['Possession %']-50)))
# dmdf['Padj Defensive Actions'] = dmdf['Padj_Tackles'] + dmdf['Padj_True Interceptions']
dmdf['Turnovers'] = dmdf['Failed Dribbles'] + dmdf['Miscontrols'] + dmdf['Dispossessions']
dmdf['Dribbles'] = dmdf['Successful Dribbles']+dmdf['Failed Dribbles']
dmdf['Dribble Success %'] = dmdf['Successful Dribbles']*100/dmdf['Dribbles']
dmdf['Aerial Challenges'] = dmdf['Aerial Challenges Lost'] + dmdf['Aerial Challenges Won']
dmdf['Aerial Win %'] = dmdf['Aerial Challenges Won']*100/dmdf['Aerial Challenges']
dmdf = dmdf.fillna(value=0)
aggdict = {'Padj_Defensive Acts':'sum','Turnovers':'sum','Aerial Challenges':'sum','Aerial Win %':'mean',
           'True Tackle Win%':'mean','Padj_Pressures':'sum','Padj_Successful Pressures':'sum',
           'Dribbles':'sum'}
groupeddefstats = dmdf.groupby(['name','team']).agg(aggdict).reset_index()
groupeddefstats

,name,team,Padj_Defensive Acts,Turnovers,Aerial Challenges,Aerial Win %,True Tackle Win%,Padj_Pressures,Padj_Successful Pressures,Dribbles
0,Aaron Mooy,Australia,37.441565,3,2,16.666667,23.148148,52.806865,8.384650,1
1,Abdalla Mahmoud El Said Bekhit,Egypt,14.222637,6,7,40.000000,27.777778,46.533785,6.577041,0
2,Abdiel Arroyo Molinar,Panama,2.151531,1,2,50.000000,16.666667,4.545831,0.000000,1
3,Abdullah Ibrahim Al Maiouf,Saudi Arabia,5.848469,0,0,0.000000,0.000000,0.000000,0.000000,0
4,Abdullah Ibrahim Otayf,Saudi Arabia,41.242734,7,1,0.000000,25.000000,60.619547,19.205832,4
...,...,...,...,...,...,...,...,...,...,...
598,İlkay Gündoğan,Germany,3.696567,2,0,0.000000,100.000000,29.572538,1.848284,0
599,Łukasz Fabiański,Poland,0.802625,0,0,0.000000,0.000000,0.000000,0.000000,0
600,Łukasz Piszczek,Poland,19.295796,8,6,62.500000,0.000000,34.109150,8.161758,1
601,Łukasz Teodorczyk,Poland,1.510163,6,4,16.666667,25.000000,10.713769,0.000000,3


#Collecting team open play shots

In [10]:
#@title

teams = df[df.player_name.isin(defenders)].team_name.unique()
teamOPSdf = df[(df.team_name.isin(teams))&(df.shot_type_name=='Open Play')].groupby('team_name').agg({'player_name':'count'}).reset_index()
teamOPSdf.rename(columns={'team_name':'team','player_name':'Open Play Shots'},inplace=True)

#Combine all aggregrate data into a single dataframe

In [14]:
#@title
timedf = pd.read_parquet(prefix+'time.parquet')
xtdf = pd.read_parquet(prefix+'xT.parquet')
xgbcdf = pd.read_parquet(prefix+'xGBC.parquet')
xgbcdf.rename(columns={'player_name':'name'},inplace=True)
pcdf = pd.read_parquet(prefix+'passcarry.parquet')
combodf = timedf[timedf.name.isin(defenders)]
combodf = combodf.merge(groupeddefstats[groupeddefstats.name.isin(defenders)],how='left')
combodf = combodf.merge(xtdf[xtdf.name.isin(defenders)],how='left')
combodf = combodf.merge(xgbcdf[xgbcdf.name.isin(defenders)],how='left')

pcdf = pcdf[pcdf.name.isin(defenders)]
pcdf['Final Third Entries'] = pcdf['Successful Final Third Passes'] + \
                                 pcdf['Final Third Carries']
pcdf['Progressive Moves'] = pcdf['Successful Progressive Passes'] + \
                               pcdf['Progressive Carries']
cols = ['Passes', 'Successful Passes','Long Passes', 'Successful Long Passes',
            'Successful Final Third Passes', 'Under Pressure Passes',
            'Successful Under Pressure Passes', 'Progressive Passes',
            'Successful Progressive Passes', 'Total Pass Length', 'Carries',
            'Pass Progressive Distance', 'Carry Distance', 'Carry Progressive Distance',
            'Final Third Entries','Progressive Moves'
        ]
pcdf = pcdf[['name']+cols]
for c in ['Successful Passes','Successful Long Passes','Successful Under Pressure Passes',
          'Successful Progressive Passes']:
    if 'Successful' in c:
        cname = ' '.join(c.split(' ')[1:])
        pcdf[cname+' Success %'] = pcdf[c]*100/pcdf[cname]
        pcdf = pcdf.drop(columns=cname)
pcdf['PPF'] = pcdf['Pass Progressive Distance']/pcdf['Total Pass Length']
pcdf['CPF'] = pcdf['Carry Progressive Distance']/pcdf['Carry Distance']
pcdf = pcdf.drop(columns=['Pass Progressive Distance','Carry Progressive Distance'])
combodf = combodf.merge(pcdf[pcdf.name.isin(defenders)],how='left')
combodf = combodf.merge(teamOPSdf,how='left')
# combodf = combodf.merge(passangles,how='left')
# combodf = combodf.merge(ppassangles,how='left')
combodf = combodf.merge(xpdf,how='left')
combodf = combodf[(combodf.time>=180)&(combodf['Open Play Passes']>=100)]
combodf['Successful Passes and Carries'] = combodf['Successful Passes'] + combodf['Carries']
for cols in ['xT', 'xT Facilitated']:
    combodf[cols] = combodf[cols]*100/combodf['Successful Passes and Carries']
for cols in ['xGBuildup', 'xGChain']:
    combodf[cols] = combodf[cols]*10/combodf['Open Play Shots']

per90cols = ['Padj_Defensive Acts','Turnovers','Aerial Challenges','Dribbles',
             'Padj_Pressures','Padj_Successful Pressures','Successful Passes',
             'Successful Long Passes','Successful Final Third Passes',
             'Successful Under Pressure Passes','Successful Progressive Passes',
             'Total Pass Length','Carries','Carry Distance',	
             'Final Third Entries','Progressive Moves','Successful Passes and Carries']

for c in per90cols:
    combodf[c] = combodf[c]/combodf['time']*90
combodf.drop(columns='Open Play Shots',inplace=True)
combodf['Turnovers per 100 Touches'] = combodf['Turnovers']*100/(combodf['Carries']+
                                                                 combodf['Dribbles'])

for c in combodf.columns.tolist()[3:]:
    combodf['Percentile '+c] = combodf[c].rank(pct = True)

combodf.head()

,name,time,team,Padj_Defensive Acts,Turnovers,Aerial Challenges,Aerial Win %,True Tackle Win%,Padj_Pressures,Padj_Successful Pressures,Dribbles,xT,xT Facilitated,xGBuildup,xGChain,Successful Passes,Successful Long Passes,Successful Final Third Passes,Successful Under Pressure Passes,Successful Progressive Passes,Total Pass Length,Carries,Carry Distance,Final Third Entries,Progressive Moves,Passes Success %,Long Passes Success %,Under Pressure Passes Success %,Progressive Passes Success %,PPF,CPF,Open Play Passes,Pass Completed Above Expected,Successful Passes and Carries,Turnovers per 100 Touches,Percentile Padj_Defensive Acts,Percentile Turnovers,Percentile Aerial Challenges,Percentile Aerial Win %,Percentile True Tackle Win%,Percentile Padj_Pressures,Percentile Padj_Successful Pressures,Percentile Dribbles,Percentile xT,Percentile xT Facilitated,Percentile xGBuildup,Percentile xGChain,Percentile Successful Passes,Percentile Successful Long Passes,Percentile Successful Final Third Passes,Percentile Successful Under Pressure Passes,Percentile Successful Progressive Passes,Percentile Total Pass Length,Percentile Carries,Percentile Carry Distance,Percentile Final Third Entries,Percentile Progressive Moves,Percentile Passes Success %,Percentile Long Passes Success %,Percentile Under Pressure Passes Success %,Percentile Progressive Passes Success %,Percentile PPF,Percentile CPF,Percentile Open Play Passes,Percentile Pass Completed Above Expected,Percentile Successful Passes and Carries,Percentile Turnovers per 100 Touches
0,Ahmed Hegazi,285,Egypt,9.577760,0.631579,1.894737,33.333333,50.000000,4.330910,0.669004,0.000000,0.106962,0.070874,0.110213,0.110213,29.368421,10.105263,1.263158,4.736842,3.473684,1089.925494,29.052632,191.405111,1.578947,3.789474,74.400000,71.111111,65.217391,39.285714,0.445737,0.517960,124,0.972886,58.421053,2.173913,0.431373,0.843137,0.401961,0.254902,0.774510,0.274510,0.215686,0.264706,0.470588,0.176471,0.352941,0.352941,0.176471,0.745098,0.166667,0.392157,0.490196,0.411765,0.294118,0.392157,0.196078,0.372549,0.088235,0.431373,0.235294,0.176471,0.647059,0.392157,0.176471,0.078431,0.215686,0.941176
5,Andreas Christensen,311,Denmark,8.372283,1.157556,1.446945,75.000000,47.500000,6.685373,1.035599,0.289389,0.067280,0.069918,0.190644,0.190644,44.565916,5.209003,2.025723,5.498392,1.736334,977.163643,38.199357,201.523577,2.025723,2.025723,92.771084,78.260870,90.476190,46.153846,0.314525,0.580109,164,1.036486,82.765273,3.007519,0.254902,1.000000,0.215686,0.774510,0.745098,0.490196,0.352941,0.627451,0.137255,0.156863,0.725490,0.686275,0.509804,0.137255,0.431373,0.568627,0.117647,0.254902,0.509804,0.470588,0.333333,0.098039,0.941176,0.735294,1.000000,0.450980,0.117647,0.666667,0.411765,0.901961,0.509804,0.960784
6,Andreas Granqvist,476,Sweden,6.699182,0.378151,1.512605,80.000000,13.333333,2.348759,0.175648,0.378151,0.184317,0.134002,0.072784,0.074888,26.470588,5.861345,2.079832,5.672269,2.836134,827.185910,23.445378,159.817139,2.647059,4.159664,81.871345,64.583333,83.333333,44.117647,0.449494,0.599963,159,1.022725,49.915966,1.587302,0.156863,0.647059,0.235294,0.901961,0.205882,0.058824,0.019608,0.764706,0.941176,0.764706,0.078431,0.098039,0.078431,0.196078,0.450980,0.607843,0.274510,0.078431,0.098039,0.235294,0.509804,0.431373,0.254902,0.274510,0.764706,0.352941,0.705882,0.784314,0.372549,0.705882,0.078431,0.823529
7,Anga Dedryck Boyata,280,Belgium,12.288028,0.000000,3.857143,40.740741,6.666667,6.718032,1.517321,0.000000,0.055122,0.064727,0.180093,0.180093,58.178571,5.464286,0.321429,4.821429,1.285714,1193.322794,52.071429,195.081015,0.321429,1.285714,94.270833,89.473684,65.217391,100.000000,0.305695,0.453728,192,1.007042,110.250000,0.000000,0.647059,0.186275,0.843137,0.372549,0.058824,0.509804,0.549020,0.264706,0.078431,0.117647,0.705882,0.647059,0.784314,0.156863,0.039216,0.450980,0.039216,0.568627,0.784314,0.431373,0.039216,0.039216,0.980392,0.980392,0.235294,1.000000,0.098039,0.176471,0.549020,0.352941,0.803922,

In [15]:
len(combodf)

51

#Save as a csv

In [16]:
combodf.to_csv('CombinedData.csv',encoding='utf-8-sig',index=False)